# Paper Figure 6: Quantification



In [1]:
import os
from test_files import get_files, prepare_test

tmp_folder = os.path.join(os.getcwd(),'temp/')
test_folder = 'fig6'

In [2]:
files = []
get_files(tmp_folder, files)
prepare_test(files, tmp_folder, test_folder)

Creating dir F:\alphapept\sandbox\temp/fig6.


In [3]:
evidence_path = 'F:/alphapept/external_test_files/PXD006109/evidence.txt'
protein_group_path = 'F:/alphapept/external_test_files/PXD006109/proteinGroups.txt'

In [ ]:
from alphapept.quantification import protein_profile_parallel_mq
import test_helper
import pandas as pd

pt = protein_profile_parallel_mq(evidence_path, protein_group_path)

C:\Users\admin\.conda\envs\alphapept\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (52,53,61) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\admin\.conda\envs\alphapept\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (120,121) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
import numpy as np
df_ = pt.copy()

ds = [_ for _ in df_.columns if 'LFQ' in _]
base_columns = [_ for _ in ds if 'HeLa2' in _]
ratio_columns = [_ for _ in ds if 'HeLa12' in _]   
to_plot = pd.DataFrame()
ref[base_columns] = df_[base_columns].replace(0, np.nan)
ref[ratio_columns] = df_[ratio_columns].replace(0, np.nan)
to_plot['Species'] = df_['Species']
to_plot['base'] = df_[base_columns].median(axis=1)
to_plot['ratio'] = df_[ratio_columns].median(axis=1)
to_plot['base_cnt'] = df_[base_columns].notna().sum(axis=1)
to_plot['ratio_cnt'] = df_[ratio_columns].notna().sum(axis=1)
to_plot['ratio_'] = np.log2(to_plot['base'] / to_plot['ratio'])
to_plot['sum_'] = np.log2(to_plot['ratio'])
valid_ap = to_plot.query(f'ratio_cnt >= 2 and base_cnt >=2')

In [ ]:
df_= ref.copy()
ds = [_ for _ in df_.columns if 'LFQ intensity' in _]
base_columns = [_ for _ in ds if '02' in _]
ratio_columns = [_ for _ in ds if '12' in _]
to_plot = pd.DataFrame()
ref[base_columns] = df_[base_columns].replace(0, np.nan)
ref[ratio_columns] = df_[ratio_columns].replace(0, np.nan)
to_plot['Species'] = df_['Species']
to_plot['base'] = df_[base_columns].median(axis=1)
to_plot['ratio'] = df_[ratio_columns].median(axis=1)
to_plot['base_cnt'] = df_[base_columns].notna().sum(axis=1)
to_plot['ratio_cnt'] = df_[ratio_columns].notna().sum(axis=1)
to_plot['ratio_'] = np.log2(to_plot['base'] / to_plot['ratio'])
to_plot['sum_'] = np.log2(to_plot['ratio'])
valid_mq = to_plot.query(f'ratio_cnt >= 2 and base_cnt >=2')

In [ ]:
def get_species(valid):

    homo = valid[valid['Species'] == 'Homo sapiens']
    e_coli = valid[valid['Species'] == 'Escherichia coli']
    
    return homo, e_coli

In [ ]:
homo_mq, e_coli_mq = get_species(valid_mq)
homo_ap, e_coli_ap = get_species(valid_ap)

In [ ]:
from matplotlib.ticker import FormatStrFormatter

hfont = {'fontname':'Arial', 'size':10}

dark_blue = '#17212b'
light_blue = '#3dc5ef'
teal= '#42dee1'
green = '#6eecb9'
yellow = '#eef5b3'

plt.figure(figsize=(2.363,2.363)) #60x60
plt.plot(homo_mq['ratio_'], homo_mq['sum_'], '.', alpha=0.5, color = light_blue, markersize=1)
plt.plot(e_coli_mq['ratio_'], e_coli_mq['sum_'], '.', alpha=0.5, color = dark_blue, markersize=1)

mean_ = homo_mq['ratio_'].mean()
std_ = homo_mq['ratio_'].std()
plt.axvline(mean_, color = 'k', linewidth=0.5)
plt.axvline(mean_+std_, linestyle=':', color='k', linewidth=0.5)
plt.axvline(mean_-std_,  linestyle=':', color='k', linewidth=0.5)

print(f'Homo {std_:.2f}')
plt.text(-2.5, 34, f'Homo \nStd \n{std_:.2f}', **hfont)

mean_ = e_coli_mq['ratio_'].mean()
std_ = e_coli_mq['ratio_'].std()
plt.axvline(mean_, color = 'k', linewidth=0.5)
plt.axvline(mean_+std_, linestyle=':', color='k', linewidth=0.5)
plt.axvline(mean_-std_,  linestyle=':', color='k', linewidth=0.5)

print(f'Ecoli {std_:.2f}')

plt.text(2.5, 34, f'E. coli \nStd \n{std_:.2f}', **hfont)

plt.xlim([-3, 5])
plt.ylim([20, 40])

ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
plt.xlabel('Ratio', **hfont)
plt.ylabel('Sum', **hfont)
plt.title('MaxQuant LFQ', **hfont)
plt.tight_layout()
plt.xticks([-3,-2,-1,0,1,2,3,4,5])
plt.savefig('p_05_mq.pdf')
plt.show()

plt.figure(figsize=(2.363,2.363)) #60x60
plt.plot(homo_ap['ratio_'], homo_ap['sum_'], '.', alpha=0.5, color = light_blue, markersize=1)
plt.plot(e_coli_ap['ratio_'], e_coli_ap['sum_'], '.', alpha=0.5, color = dark_blue, markersize=1)



mean_ = homo_ap['ratio_'].mean()
std_ = homo_ap['ratio_'].std()
plt.axvline(mean_, color = 'k', linewidth=0.5)
plt.axvline(mean_+std_, linestyle=':', color='k', linewidth=0.5)
plt.axvline(mean_-std_,  linestyle=':', color='k', linewidth=0.5)

plt.text(-2.5, 34, f'Homo \nStd \n{std_:.2f}', **hfont)

print(f'Std {std_:.2f}')

mean_ = e_coli_ap['ratio_'].mean()
std_ = e_coli_ap['ratio_'].std()
plt.axvline(mean_, color = 'k', linewidth=0.5)
plt.axvline(mean_+std_, linestyle=':', color='k', linewidth=0.5)
plt.axvline(mean_-std_,  linestyle=':', color='k', linewidth=0.5)

print(f'Ecoli {std_:.2f}')

plt.text(2.5, 34, f'E. coli \nStd \n{std_:.2f}', **hfont)


plt.xlim([-3, 5])
plt.ylim([20, 40])

ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
plt.xlabel('Ratio', **hfont)
plt.ylabel('Sum', **hfont)
plt.title('AlphaPept LFQ', **hfont)
plt.tight_layout()
plt.xticks([-3,-2,-1,0,1,2,3,4,5])
plt.savefig('p_05_ap.pdf')
plt.show()

In [ ]:
from itertools import combinations
from numba.typed import List

import alphapept.quantification

n_sequences = 10
n_samples = 6

column_combinations = List()
[column_combinations.append(_) for _ in combinations(range(n_samples), 2)]

methods = {'L-FBGS-B':alphapept.quantification.solve_profile_LFBGSB, 'SLSQP':alphapept.quantification.solve_profile_SLSQP, 'trf':alphapept.quantification.solve_profile_trf}
results = []

for run in range(20):
    signal, sample, ground_truth = alphapept.quantification.generate_dummy_data(n_sequences, n_samples)
    ratios = alphapept.quantification.get_protein_ratios(signal.values, column_combinations)

    for method in methods.keys():
        error = False
        start = time()
        try:
            solution, success = methods[method](ratios)
            mape = np.mean(np.abs((solution-ground_truth)/ground_truth))
        except:
            error = True
            mape = np.nan
            
        
        end = time()

        time_elapsed_s = (end-start)

        results.append((method, time_elapsed_s, mape, error, success))
    
result_df = pd.DataFrame(results, columns=['Method', 'Time Elapsed (s)', 'Mean absolute percentage error', 'Errors', 'Success'])

result_df.groupby('Method').mean()